Rough Draft for the take home final. 

In [ ]:
!pip install xmltodict

In [ ]:
rates = ["EUR", "GBP", "USD", "DZD", "AUD", "BWP", "BND", "CAD", "CLP", "CNY", "COP", "CZK", "DKK", "HUF", "ISK", "INR", "IDR", "ILS", "KZT", "KRW", "KWD", "LYD", "MYR", "MUR", "NPR", "NZD", "NOK", "OMR", "PKR", "PLN", "QAR", "RUB", "SAR", "SGD", "ZAR", "LKR", "SEK", "CHF", "THB", "TTD"]
ratesForBase = [r for r in rates if r != "USD" and r != "EUR" and r != "GBP"]

Code Given by Professor Powers in the Prompt. 

In [ ]:
import requests
import xmltodict
import json
import random

# URL of the XML data
date = "2011-05-04"
base = random.choice(ratesForBase)
url = f"https://www.floatrates.com/historical-exchange-rates.html?operation=rates&pb_id=1775&page=historical&currency_date={date}&base_currency_code={base}&format_type=xml"
print(url)
# Fetch the XML data
response = requests.get(url)
response.raise_for_status()  # Ensure we notice bad responses

# Parse the XML data to a Python dictionary
data_dict = xmltodict.parse(response.text)

# Convert the dictionary to a JSON string
json_data = json.dumps(data_dict, indent=4)

# Print the JSON data
print(json_data)

# Optionally, write the JSON data to a file
with open(f"{date}_exchange_rates_{base}.json", "w") as json_file:
    json_file.write(json_data)

In [ ]:
import os
import requests 
import xmltodict
import json
import random
from datetime import datetime, timedelta
import threading
import time

rates = ["EUR", "GBP", "USD", "DZD", "AUD", "BWP", "BND", "CAD", "CLP", "CNY", "COP",
         "CZK", "DKK", "HUF", "ISK", "INR", "IDR", "ILS", "KZT", "KRW", "KWD", "LYD",
         "MYR", "MUR", "NPR", "NZD", "NOK", "OMR", "PKR", "PLN", "QAR", "RUB", "SAR",
         "SGD", "ZAR", "LKR", "SEK", "CHF", "THB", "TTD"]

rates_for_base = [r for r in rates if r not in ["USD", "EUR", "GBP"]]
base = random.choice(rates_for_base)


start = datetime(2011, 5, 4)
end = datetime.today()
dates = []
while start <= end:
    dates.append(start.strftime("%Y-%m-%d"))
    start += timedelta(days=1)

folder = "data_" + base
os.makedirs(folder, exist_ok=True)

def download(date):
    path = f"{folder}/{date}.json"
    if os.path.exists(path):
        return
    url = f"https://www.floatrates.com/historical-exchange-rates.html?operation=rates&pb_id=1775&page=historical&currency_date={date}&base_currency_code={base}&format_type=xml"
    try:
        r = requests.get(url, timeout=10)
        d = xmltodict.parse(r.text)
        with open(path, "w") as f:
            json.dump(d, f)
        print("Saved", date)
    except:
        print("Failed", date)

def run_all():
    i = 0
    while i < len(dates):
        group = []
        for j in range(10):
            if i + j < len(dates):
                t = threading.Thread(target=download, args=(dates[i + j],))
                t.start()
                group.append(t)
        for t in group:
            t.join()
        i += 10

print("Starting", base)
run_all()
print("Done")

In [ ]:
import threading
from downloader import download_data
from utils import get_all_dates, pick_random_base, already_downloaded

dates = get_all_dates("2011-05-04")
base = pick_random_base()

threads = []

def download_for_data(date):
    if not already_downloaded(date, base):
        print("Getting:", date)
        download_data(date, base)
    else:
        print("Already have:", date)

for date in dates:
    t = threading.Thread(target= download_for_data, args=(date,))
    threads.append(t)
    t.start()
    if len(threads) >= 10:
        for t in threads:
            t.join()
        threads = []

for t in threads:
    t.join()

In [ ]:
import requests
import xmltodict
import time
from saver import save_json
from utils import make_url, make_filename

def download_data(date, base):
    url = make_url(date, base)
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = xmltodict.parse(response.text)
            filename = make_filename(date, base)
            save_json(data, filename)
        else:
            print("Error getting data for", date)
    except:
        print("Problem getting data for", date)
    time.sleep(0.5)

In [ ]:
import os
import json

def save_json(data, filename):
    folder = os.path.dirname(filename)
    if not os.path.exists(folder):
        os.makedirs(folder)
    with open(filename, "w") as f:
        json.dump(data, f)

In [ ]:
import random
import os
from datetime import datetime, timedelta

rates = ["EUR", "GBP", "USD", "DZD", "AUD", "BWP", "BND", "CAD", "CLP", "CNY",
         "COP", "CZK", "DKK", "HUF", "ISK", "INR", "IDR", "ILS", "KZT", "KRW",
         "KWD", "LYD", "MYR", "MUR", "NPR", "NZD", "NOK", "OMR", "PKR", "PLN",
         "QAR", "RUB", "SAR", "SGD", "ZAR", "LKR", "SEK", "CHF", "THB", "TTD"]

def pick_random_base():
    choices = []
    for r in rates:
        if r != "USD" and r != "EUR" and r != "GBP":
            choices.append(r)
    return random.choice(choices)

def get_all_dates(start):
    dates = []
    today = datetime.today()
    d = datetime.strptime(start, "%Y-%m-%d")
    while d <= today:
        dates.append(d.strftime("%Y-%m-%d"))
        d += timedelta(days=1)
    return dates

def make_url(date, base):
    return "https://www.floatrates.com/historical-exchange-rates.html?operation=rates&pb_id=1775&page=historical&currency_date=" + date + "&base_currency_code=" + base + "&format_type=xml"

def make_filename(date, base):
    return "data/" + base + "/" + date + "_exchange_rates.json"

def already_downloaded(date, base):
    file_path = make_filename(date, base)
    return os.path.exists(file_path)